***
### Figure 3.20 Qualitative Visual Assessment
***

In [ ]:
# Import of required libraries--------------------------------------------------
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import tensorflow as tf
from tensorflow.keras.models import load_model
from traffic.core import Traffic

# Import of trajectory data-----------------------------------------------------
t = Traffic.from_file("/mnt/beegfs/store/krum/MT/encoded_scaled_split/t_test.parquet")


# Loading of model and scalers--------------------------------------------------
def rmse_lat():
    return None


def rmse_lon():
    return None


def rmse_alt():
    return None


model = load_model(
    f"/home/krum/git/MT_krum_code/models/robust-glitter-237.keras",
    custom_objects={
        "rmse_lat": rmse_lat,
        "rmse_lon": rmse_lon,
        "rmse_alt": rmse_alt,
        "weighted_mse": tf.keras.losses.MeanSquaredError(),
    },
)

with open(
    "/mnt/beegfs/store/krum/MT/encoded_scaled_split/scaler_in.pkl",
    "rb",
) as file:
    scaler_in = pickle.load(file)

with open(
    "/mnt/beegfs/store/krum/MT/encoded_scaled_split/scaler_out.pkl",
    "rb",
) as file:
    scaler_out = pickle.load(file)

# Flight selection--------------------------------------------------------------
f = t[100]

# Prediction--------------------------------------------------------------------
# Time variant inputs unscaled
f_in_var_unscaled = f.data[
    [
        "latitude",
        "longitude",
        "altitude",
        "wind_x_2min_avg",
        "wind_y_2min_avg",
        "temperature_gnd",
        "humidity_gnd",
        "pressure_gnd",
    ]
]

# Time variant inputs scaled
f_in_var = f.data[
    [
        "latitude_scaled",
        "longitude_scaled",
        "altitude_scaled",
        "wind_x_2min_avg_scaled",
        "wind_y_2min_avg_scaled",
        "temperature_gnd_scaled",
        "humidity_gnd_scaled",
        "pressure_gnd_scaled",
    ]
]

# Time invariant inputs
f_in_con = f.data[
    [
        "toff_weight_kg_scaled",
        "typecode_A20N",
        "typecode_A21N",
        "typecode_A319",
        "typecode_A320",
        "typecode_A321",
        "typecode_A333",
        "typecode_A343",
        "typecode_B77W",
        "typecode_BCS1",
        "typecode_BCS3",
        "typecode_CRJ9",
        "typecode_DH8D",
        "typecode_E190",
        "typecode_E195",
        "typecode_E290",
        "typecode_E295",
        "typecode_F100",
        "typecode_SB20",
        "SID_DEGES",
        "SID_GERSA",
        "SID_VEBIT",
        "SID_ZUE",
        "hour_sin",
        "hour_cos",
        "weekday_sin",
        "weekday_cos",
        "month_sin",
        "month_cos",
    ]
]

# True outputs
f_out_unscaled = f.data[
    [
        "latitude",
        "longitude",
        "altitude",
    ]
]

# Variable inputs unscaled
inputs_var_unscaled = []
for i in range(len(f_in_var_unscaled) - 10 - 180):
    inputs_var_unscaled.append(f_in_var_unscaled.iloc[i : i + 10].to_numpy())
flattened_input = [item for sublist in inputs_var_unscaled for item in sublist]
input_var_unscaled = np.stack(flattened_input).reshape(-1, 10, 8)

# Variable inputs scaled
inputs_var = []
for i in range(len(f_in_var) - 10 - 180):
    inputs_var.append(f_in_var.iloc[i : i + 10].to_numpy())

flattened_input = [item for sublist in inputs_var for item in sublist]
input_var = np.stack(flattened_input).reshape(-1, 10, 8)

# Constant inputs
inputs_con = []
for i in range(len(f_in_con) - 10 - 180):
    inputs_con.append(f_in_con.iloc[i + 10].to_numpy())
flattened_input = [item for sublist in inputs_con for item in sublist]
input_con = np.stack(flattened_input).reshape(-1, 1, 29)

# True_outputs unscaled
outputs_true = []

for i in range(len(f_out_unscaled) - 10 - 180):
    outputs_true.append(f_out_unscaled.iloc[i + 15 : i + 15 + 180 : 5].to_numpy())

flattened_input = [item for sublist in outputs_true for item in sublist]
output_true = np.stack(flattened_input).reshape(-1, 36, 3)

# Prediction and unscaling
output = model.predict((input_var, input_con))
output_unscaled = scaler_out.inverse_transform(output.reshape(-1, 3)).reshape(
    -1, 37, 3
)[:, 1:, :]

# Generation of dataframe for plotting------------------------------------------
# Actual trajectory
df_full = f.data

# Input output pairs
input_output = []

# For each timestamp generate a dataframe and append to list
for i in range(input_var.shape[0]):
    lat_in = input_var_unscaled[i, :, 0]
    lon_in = input_var_unscaled[i, :, 1]
    alt_in = input_var_unscaled[i, :, 2]

    lat_out = output_unscaled[i, :, 0]
    lon_out = output_unscaled[i, :, 1]
    alt_out = output_unscaled[i, :, 2]

    lat_out_true = output_true[i, :, 0]
    lon_out_true = output_true[i, :, 1]
    alt_out_true = output_true[i, :, 2]

    # Model input
    df1 = pd.DataFrame(
        {
            "latitude": lat_in,
            "longitude": lon_in,
            "altitude": alt_in,
            "color": "#636efa",
        }
    )

    # True output
    df2 = pd.DataFrame(
        {
            "latitude": lat_out_true,
            "longitude": lon_out_true,
            "altitude": alt_out_true,
            "color": "grey",
        }
    )

    # Model prediction
    df3 = pd.DataFrame(
        {
            "latitude": lat_out,
            "longitude": lon_out,
            "altitude": alt_out,
            "color": "#00cc96",
        }
    )

    # Concatenate input and output and add to list
    df = pd.concat([df1, df2, df3])
    input_output.append(df)

# Simulation length (timestamps)
lengths = len(df_full) - 180 - 10

# Generate plot-----------------------------------------------------------------
fig = make_subplots(
    rows=2,
    cols=1,
    specs=[[{"type": "scattermapbox"}], [{}]],
    row_heights=[0.6, 0.4],
    subplot_titles=("Position", "Altitude"),
    vertical_spacing=0.07,
)

# Add initial traces
# Full trajectory position
fig.append_trace(
    go.Scattermapbox(
        mode="lines",
        lat=df_full["latitude"],
        lon=df_full["longitude"],
        line=dict(width=4, color="lightgrey"),
        name="Full trajectory",
        showlegend=False,
    ),
    row=1,
    col=1,
)

# Model input-output position
fig.append_trace(
    go.Scattermapbox(
        lat=input_output[0]["latitude"],
        lon=input_output[0]["longitude"],
        marker=dict(size=7, color=input_output[0]["color"]),
        mode="markers",
        showlegend=False,
    ),
    row=1,
    col=1,
)

# Full trajectory altitude
fig.append_trace(
    go.Scatter(
        x=np.arange(0, len(df_full)),
        y=df_full["altitude"],
        line=dict(width=4, color="lightgrey"),
        mode="lines",
        name="full",
        showlegend=False,
    ),
    row=2,
    col=1,
)

# Input altitude
fig.append_trace(
    go.Scatter(
        x=np.arange(0, len(df_full)),
        y=input_output[0].iloc[0:10]["altitude"],
        marker=dict(size=6, color=input_output[0].iloc[0:10]["color"]),
        mode="markers",
        name="full",
        showlegend=False,
    ),
    row=2,
    col=1,
)

# True altitude
fig.append_trace(
    go.Scatter(
        x=np.arange(15, len(df_full), 5),
        y=input_output[0].iloc[10:46]["altitude"],
        marker=dict(size=6, color=input_output[0].iloc[10:46]["color"]),
        mode="markers",
        name="full",
        showlegend=False,
    ),
    row=2,
    col=1,
)

# Predicted altitude
fig.append_trace(
    go.Scatter(
        x=np.arange(15, len(df_full), 5),
        y=input_output[0].iloc[46:]["altitude"],
        marker=dict(size=6, color=input_output[0].iloc[46:]["color"]),
        mode="markers",
        name="full",
        showlegend=False,
    ),
    row=2,
    col=1,
)

# Update Figure layout
fig.update_xaxes(
    title_text="Elapsed time [s]",
    range=[0, len(df_full)],
    row=2,
    col=1,
    title_font={"size": 25},
    tickfont={"size": 20},
)

fig.update_yaxes(
    title_text="Baroaltitude [ft]",
    range=[df_full.altitude.min(), df_full.altitude.max()],
    row=2,
    col=1,
    title_font={"size": 25},
    tickfont={"size": 20},
)

fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=10,
        center=dict(
            lat=np.mean(df_full["latitude"].mean()),
            lon=np.mean(df_full["longitude"].mean()),
        ),
    ),
    width=1600,
    height=1000,
    margin=dict(l=50, r=20, t=40, b=40),
)

# Animation---------------------------------------------------------------------
# Creation of animation frames
frames = [
    go.Frame(
        data=[
            # Position
            go.Scattermapbox(
                lat=input_output[k]["latitude"],
                lon=input_output[k]["longitude"],
                marker=dict(size=7, color=input_output[k]["color"]),
                mode="markers",
                showlegend=False,
            ),
            # Altitude input
            go.Scatter(
                x=np.arange(k, k + 180),
                y=input_output[k].iloc[0:10]["altitude"],
                marker=dict(size=6, color=input_output[k].iloc[0:10]["color"]),
                mode="markers",
                showlegend=False,
            ),
            # Altitude true output
            go.Scatter(
                x=np.arange(k + 15, k + 15 + 180, 5),
                y=input_output[k].iloc[10:46]["altitude"],
                marker=dict(size=6, color=input_output[k].iloc[10:46]["color"]),
                mode="markers",
                showlegend=False,
            ),
            # Altitude true output
            go.Scatter(
                x=np.arange(k + 15, k + 15 + 180, 5),
                y=input_output[k].iloc[46:]["altitude"],
                marker=dict(size=6, color=input_output[k].iloc[46:]["color"]),
                mode="markers",
                showlegend=False,
            ),
        ],
        # Frame names
        name=f"fr{k}",
        traces=[
            1,
            3,
            4,
            5,
            6,
        ],
    )
    # Loop over all timestamps
    for k in range(lengths)
]

# Add frames to the figure
fig.update(frames=frames)


# Animation frame arguments
def frame_args(duration):
    return {
        "frame": {"duration": duration},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "linear"},
    }


# Animation frame duration
fr_duration = 750

# Slider configuration
sliders = [
    {
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "font": {"size": 20},
        "steps": [
            {
                "args": [[f.name], frame_args(fr_duration)],
                "label": f"{k+1}s",
                "method": "animate",
            }
            for k, f in enumerate(fig.frames)
        ],
    }
]

# Update of slider and button layout
fig.update_layout(
    sliders=sliders,
    updatemenus=[
        {
            "buttons": [
                {
                    "args": [None, frame_args(fr_duration)],
                    "label": "&#9654;",  # play symbol
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(fr_duration)],
                    "label": "&#9724;",  # pause symbol
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }
    ],
)

for annotation in fig["layout"]["annotations"]:
    annotation["font"] = dict(size=30)

fig.show()

***
### Figure 3.21 Take-off distance trajectory example
***

In [ ]:
# Import of required libraries--------------------------------------------------
import pickle

import plotly.graph_objects as go
import tensorflow as tf

from tensorflow.keras.models import load_model
from traffic.core import Traffic

# Import of trajectory data-----------------------------------------------------
t = Traffic.from_file("/mnt/beegfs/store/krum/MT/encoded_scaled_split/t_test.parquet")


# Loading the model and scaler--------------------------------------------------
def rmse_lat():
    return None


def rmse_lon():
    return None


def rmse_alt():
    return None


model = load_model(
    f"/home/krum/git/MT_krum_code/models/snowy-gorge-126.keras",
    custom_objects={
        "rmse_lat": rmse_lat,
        "rmse_lon": rmse_lon,
        "rmse_alt": rmse_alt,
        "weighted_mse": tf.keras.losses.MeanSquaredError(),
    },
)

with open(
    "/mnt/beegfs/store/krum/MT/encoded_scaled_split/scaler_out.pkl",
    "rb",
) as file:
    scaler_out = pickle.load(file)

# Selection of flight-----------------------------------------------------------
flight = t[1]

# Prediction of the model-------------------------------------------------------
f_in_var_unscaled = flight.data[
    [
        "latitude",
        "longitude",
        "altitude",
    ]
].iloc[0:10]

input_var = (
    flight.data[
        [
            "latitude_scaled",
            "longitude_scaled",
            "altitude_scaled",
            "wind_x_2min_avg_scaled",
            "wind_y_2min_avg_scaled",
            "temperature_gnd_scaled",
            "humidity_gnd_scaled",
            "pressure_gnd_scaled",
        ]
    ]
    .iloc[0:10]
    .to_numpy()
    .reshape(1, 10, 8)
)

input_con = (
    flight.data[
        [
            "toff_weight_kg_scaled",
            "typecode_A20N",
            "typecode_A21N",
            "typecode_A319",
            "typecode_A320",
            "typecode_A321",
            "typecode_A333",
            "typecode_A343",
            "typecode_B77W",
            "typecode_BCS1",
            "typecode_BCS3",
            "typecode_CRJ9",
            "typecode_DH8D",
            "typecode_E190",
            "typecode_E195",
            "typecode_E290",
            "typecode_E295",
            "typecode_F100",
            "typecode_SB20",
            "SID_DEGES",
            "SID_GERSA",
            "SID_VEBIT",
            "SID_ZUE",
            "hour_sin",
            "hour_cos",
            "weekday_sin",
            "weekday_cos",
            "month_sin",
            "month_cos",
        ]
    ]
    .iloc[10]
    .to_numpy()
    .reshape(1, 1, 29)
)

output = model.predict((input_var, input_con), verbose=0)
output_unscaled = scaler_out.inverse_transform(output.reshape(-1, 3)).reshape(37, 3)[
    1:, :
]

# Generate plot-----------------------------------------------------------------
fig = go.Figure()

# Actual trajectory
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=flight.data["latitude"],
        lon=flight.data["longitude"],
        line=dict(width=4, color="lightgrey"),
        name="Actual trajectory",
        showlegend=True,
    ),
)

# Model input
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=flight.data["latitude"].iloc[0:10],
        lon=flight.data["longitude"].iloc[0:10],
        marker=dict(size=10, color="#636efa"),
        name="Model input",
        showlegend=True,
    ),
)

# Prediction
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=output_unscaled[:, 0],
        lon=output_unscaled[:, 1],
        marker=dict(size=10, color="#00cc96"),
        name="Prediction",
        showlegend=True,
    ),
)

# Layout
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=10,
        center=dict(
            lat=flight.data["latitude"].mean(),
            lon=flight.data["longitude"].mean(),
        ),
    ),
    width=1100,
    height=500,
    margin=dict(l=5, r=5, t=5, b=5),
    legend=dict(font=dict(size=25)),
)

fig.show()

***
### Figure 3.22 Take-off detection
***

In [ ]:
# Import of required libraries--------------------------------------------------
import pickle

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import tensorflow as tf

from geopy.distance import distance
from tensorflow.keras.models import load_model
from traffic.core import Traffic

# Import of trajectory data-----------------------------------------------------
t = Traffic.from_file("/mnt/beegfs/store/krum/MT/encoded_scaled_split/t_test.parquet")


# Loading the model and scaler--------------------------------------------------
def rmse_lat():
    return None


def rmse_lon():
    return None


def rmse_alt():
    return None


model = load_model(
    f"/home/krum/git/MT_krum_code/models/snowy-gorge-126.keras",
    custom_objects={
        "rmse_lat": rmse_lat,
        "rmse_lon": rmse_lon,
        "rmse_alt": rmse_alt,
        "weighted_mse": tf.keras.losses.MeanSquaredError(),
    },
)

with open(
    "/mnt/beegfs/store/krum/MT/encoded_scaled_split/scaler_out.pkl",
    "rb",
) as file:
    scaler_out = pickle.load(file)


# Function to calculate distance between rows of coordinates--------------------
def calculate_distance(row1, row2):
    coords_1 = (row1["latitude"], row1["longitude"])
    coords_2 = (row2["latitude"], row2["longitude"])
    return distance(coords_1, coords_2).meters


# Selection of flight-----------------------------------------------------------
flight = t[564]

# Determine actual take-off distance--------------------------------------------
flightdata = flight.cumulative_distance().data
flightdata["cumdist_m"] = flightdata["cumdist"] * 1852
runalt_m = ((flightdata[0:10].altitude) / 3.281).mean()
flightdata["alt_agl_m"] = (flightdata.altitude / 3.281) - runalt_m
flightdata["status"] = np.where(flightdata["alt_agl_m"] >= 50, "air", "ground")
to_dist_actual = round(flightdata[flightdata["status"] == "air"]["cumdist_m"].iloc[0])
to_dist_actual_alt = round(
    flightdata[flightdata["status"] == "air"]["alt_agl_m"].iloc[0]
)

# Generate Trajectory prediction------------------------------------------------
f_in_var_unscaled = flight.data[
    [
        "latitude",
        "longitude",
        "altitude",
    ]
].iloc[0:10]

input_var = (
    flight.data[
        [
            "latitude_scaled",
            "longitude_scaled",
            "altitude_scaled",
            "wind_x_2min_avg_scaled",
            "wind_y_2min_avg_scaled",
            "temperature_gnd_scaled",
            "humidity_gnd_scaled",
            "pressure_gnd_scaled",
        ]
    ]
    .iloc[0:10]
    .to_numpy()
    .reshape(1, 10, 8)
)

input_con = (
    flight.data[
        [
            "toff_weight_kg_scaled",
            "typecode_A20N",
            "typecode_A21N",
            "typecode_A319",
            "typecode_A320",
            "typecode_A321",
            "typecode_A333",
            "typecode_A343",
            "typecode_B77W",
            "typecode_BCS1",
            "typecode_BCS3",
            "typecode_CRJ9",
            "typecode_DH8D",
            "typecode_E190",
            "typecode_E195",
            "typecode_E290",
            "typecode_E295",
            "typecode_F100",
            "typecode_SB20",
            "SID_DEGES",
            "SID_GERSA",
            "SID_VEBIT",
            "SID_ZUE",
            "hour_sin",
            "hour_cos",
            "weekday_sin",
            "weekday_cos",
            "month_sin",
            "month_cos",
        ]
    ]
    .iloc[10]
    .to_numpy()
    .reshape(1, 1, 29)
)

output = model.predict((input_var, input_con), verbose=0)
output_unscaled = scaler_out.inverse_transform(output.reshape(-1, 3)).reshape(37, 3)

# Combine input and output into one dataframe-----------------------------------
# Input
df_in = f_in_var_unscaled

# Output
df_out = pd.DataFrame(output_unscaled, columns=["latitude", "longitude", "altitude"])
df_out["altitude"].iloc[0] = df_in["altitude"].iloc[-1]
df_out["latitude"].iloc[0] = df_in["latitude"].iloc[-1]
df_out["longitude"].iloc[0] = df_in["longitude"].iloc[-1]
df_out["altitude"].iloc[1] = 1180
df_out["Time"] = [i * 5 for i in range(len(df_out))]
df_out.set_index("Time", inplace=True)
time_index = pd.RangeIndex(start=0, stop=df_out.index[-1] + 1, step=1)
df_out = df_out.reindex(time_index)
df_out = df_out.interpolate(method="linear")
df_out.reset_index(inplace=True)
df_out.rename(columns={"index": "Time"}, inplace=True)

# Combine input and output
df = pd.concat([df_in, df_out]).reset_index(drop=True)

# Change to meters agl
df["alt_agl_m"] = (df.altitude / 3.281) - runalt_m

# Calculate distance travelled--------------------------------------------------
dist_diff = df.apply(
    lambda row: (calculate_distance(df.loc[row.name - 1], row) if row.name > 0 else 0),
    axis=1,
)
df["cumdist"] = dist_diff.cumsum()

# Determine predicted take-off distance-----------------------------------------
to_dist_pred = round(df[df["alt_agl_m"] >= 50]["cumdist"].iloc[0])
to_dist_pred_alt = round(df[df["alt_agl_m"] >= 50]["alt_agl_m"].iloc[0])

# Generate plot-----------------------------------------------------------------
fig = go.Figure()

# Actual trajectory
fig.add_trace(
    go.Scatter(
        x=flightdata["cumdist_m"].iloc[0:110],
        y=flightdata["alt_agl_m"].iloc[0:110],
        mode="markers + lines",
        name="Actual trajectory",
        line=dict(color="darkgrey"),
        marker=dict(color="darkgrey"),
    )
)

# Model input
fig.add_trace(
    go.Scatter(
        x=flightdata["cumdist_m"].iloc[0:10],
        y=flightdata["alt_agl_m"].iloc[0:10],
        mode="markers + lines",
        name="Model input",
        line=dict(color="#636efa"),
        marker=dict(color="#636efa"),
    )
)

# Model prediction
fig.add_trace(
    go.Scatter(
        x=df["cumdist"].iloc[9:100],
        y=df["alt_agl_m"].iloc[9:100],
        mode="markers + lines",
        name="Predicted",
        line=dict(color="#00cc96"),
        marker=dict(color="#00cc96"),
    )
)

# Horizontal line at y = 50m
fig.add_shape(
    dict(
        type="line",
        x0=0,
        y0=50,
        x1=8000,
        y1=50,
        line=dict(color="black", width=1, dash="dash"),
    )
)

# Vertical line at x = to_dist_actual
fig.add_shape(
    dict(
        type="line",
        x0=to_dist_actual,
        y0=0,
        x1=to_dist_actual,
        y1=to_dist_actual_alt,
        line=dict(color="darkgrey", width=2),
    )
)

# Vertical line at x = to_dist_pred
fig.add_shape(
    dict(
        type="line",
        x0=to_dist_pred,
        y0=0,
        x1=to_dist_pred,
        y1=to_dist_pred_alt,
        line=dict(color="#00cc96", width=2),
    )
)

# Add annotations for take-off distances
fig.add_annotation(
    x=to_dist_actual,
    y=-10,
    text=to_dist_actual,
    showarrow=False,
    font=dict(size=18, color="darkgrey"),
)
fig.add_annotation(
    x=to_dist_pred,
    y=-10,
    text=to_dist_pred,
    showarrow=False,
    font=dict(size=18, color="#00cc96"),
)

# Layout
fig.update_layout(
    width=1100,
    height=500,
    margin=dict(l=5, r=5, t=5, b=5),
    xaxis_title="Cumulative distance travelled [m]",
    yaxis_title="Altitude above runway level [m]",
    xaxis_title_font=dict(size=25),
    yaxis_title_font=dict(size=25),
    xaxis=dict(tickfont=dict(size=20), title_standoff=30),
    yaxis=dict(tickfont=dict(size=20), title_standoff=30),
    legend=dict(font=dict(size=25)),
)

***
### Figure 3.23 Out-of-sample
***

In [ ]:
# Import of required libraries--------------------------------------------------
from traffic.core import Traffic
from traffic.data import airports
import numpy as np
import plotly.graph_objects as go

# Import of trajectory data-----------------------------------------------------
t = Traffic.from_file("/mnt/beegfs/store/krum/MT/encoded_scaled_split/t_test.parquet")


# Function required to draw a circle around a point-----------------------------
def circle_coordinates(lat: float, lon: float, radius_km: float, num_points: int = 360):
    """
    Calculate the coordinates forming a circle of a given diameter around a
    centre coordinate.

    Parameters
    ----------
    lat : float
        Latitude of the center point.
    lon : float
        Longitude of the center point.
    radius_km : float
        Radius of the circle in kilometers.
    num_points : int, optional
        Number of points to generate the circle, by default 360.
    """

    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    R = 6371.0
    angles = np.linspace(0, 2 * np.pi, num_points)
    circle_lat = []
    circle_lon = []

    for angle in angles:
        lat_point = np.arcsin(
            np.sin(lat_rad) * np.cos(radius_km / R)
            + np.cos(lat_rad) * np.sin(radius_km / R) * np.cos(angle)
        )
        lon_point = lon_rad + np.arctan2(
            np.sin(angle) * np.sin(radius_km / R) * np.cos(lat_rad),
            np.cos(radius_km / R) - np.sin(lat_rad) * np.sin(lat_point),
        )
        circle_lat.append(np.degrees(lat_point))
        circle_lon.append(np.degrees(lon_point))

    return circle_lat, circle_lon


# Generate the coordinates representing the circle of 30 nm around LSZH---------
circle_lat, circle_lon = circle_coordinates(
    airports["LSZH"].latitude, airports["LSZH"].longitude, 30 * 1.852, 360
)

# Selection of flight-----------------------------------------------------------
x = "BEL2LX_919874"

# Generate plot 1---------------------------------------------------------------
fig = go.Figure()

# Circle
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=circle_lat,
        lon=circle_lon,
        line=dict(color="black"),
        showlegend=True,
        name="30 NM radius",
    )
)

# Input
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=t[x].data[-190:-180].latitude,
        lon=t[x].data[-190:-180].longitude,
        marker=dict(
            color="#636efa",
            size=8,
        ),
        showlegend=True,
        name="Input",
    )
)

# Output
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=t[x].data[-180:-1].latitude.to_numpy()[1::5],
        lon=t[x].data[-180:-1].longitude.to_numpy()[1::5],
        marker=dict(
            color="#00cc96",
            size=8,
        ),
        showlegend=True,
        name="True output",
    )
)

# Layout
fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=8.3,
        center=dict(lat=airports["LSZH"].latitude, lon=airports["LSZH"].longitude),
    ),
    legend=dict(font=dict(size=25)),
)
fig.show()

# Generate plot 2---------------------------------------------------------------
fig = go.Figure()

# Circle
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=circle_lat,
        lon=circle_lon,
        line=dict(color="black"),
        showlegend=True,
        name="30 NM radius",
    )
)

# Input
fig.add_trace(
    go.Scattermapbox(
        mode="markers",
        lat=t["BEL2LX_919874"].data.latitude[-11:-1],
        lon=t["BEL2LX_919874"].data.longitude[-11:-1],
        marker=dict(
            color="#636efa",
            size=8,
        ),
        showlegend=True,
        name="Input",
    )
)

# Layout
fig.update_layout(
    width=1000,
    height=800,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=8.3,
        center=dict(lat=airports["LSZH"].latitude, lon=airports["LSZH"].longitude),
    ),
    legend=dict(font=dict(size=25)),
)
fig.show()